In [61]:
import xmltodict, json, feedparser
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time, re
import pandas as pd
from collections import defaultdict
import json
from datetime import datetime
from selenium.webdriver.common.by import By
from pprint import pprint
import dataframe_image as dfi
import os, sys
from PIL import Image

In [77]:
title = '투자설명서'
link = 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220919000002'
companyName = '모델솔루션'
text = title
date = '2002-09-26'

if ('투자설명서' in title) and not('정정' in title):

  option = Options()
  # option.add_argument('headless')
  driver = webdriver.Chrome(options=option)
  driver.get(link)
  # time.sleep(2)

  key_word = '투자위험요소'
  driver.find_element(By.PARTIAL_LINK_TEXT, key_word).click()

  driver.current_url
  html_source = driver.page_source
  bk, key_word, ak = html_source.partition(key_word)

  rcpNo = re.findall(r'rcpNo\'] = "(.*?)";\n', ak)
  dcmNo = re.findall(r'dcmNo\'] = "(.*?)";\n', ak)
  eleId = re.findall(r'eleId\'] = "(.*?)";\n', ak)
  offset = re.findall(r'offset\'] = "(.*?)";\n', ak)
  length = re.findall(r'length\'] = "(.*?)";\n', ak)
  dtd = re.findall(r'dtd\'] = "(.*?)";\n', ak)

  # url이 있어야 read_html을 통해 table 가져올 수 있다
  frame_url = 'https://dart.fss.or.kr/report/viewer.do?rcpNo=' + rcpNo[0] + '&dcmNo=' + dcmNo[0] + '&eleId=' + eleId[0] + '&offset=' + offset[0] + '&length=' + length[0] + '&dtd=' + dtd[0]
  driver.get(frame_url)

  # html_source = driver.page_source
  table_ifrm = pd.read_html(frame_url, index_col=0, header=0)

  for no in range(len(table_ifrm)):
    for index, row in table_ifrm[no].iterrows():
      try:
        if '벤처금융' in index:
          tableOverhang = table_ifrm[no]
      except:
        pass
  
  dfi.export(tableOverhang, 'images/table_overhang.png')
  im = Image.open('images/table_overhang.png')
  im.thumbnail((998, 680), Image.ANTIALIAS)
  im.save('images/table_overhang.png')

  try:
    with open('IPODB.json', encoding='utf-8') as db:
      res = json.load(db)

    seen = set()
    for date in res.keys():
      for company in res[date].keys():
        seen.add((date, company, res[date][company]['내용'], res[date][company]['링크']))

    # seen = {(date, company, res[date][company]['내용'], res[date][company]['링크']) for date in res.keys() for company in res[date].keys()} # 중첩된 집합 표기법?

    link = 'images/table_overhang.png'

    def add_entry(res, date, companyName, text, link):

        # check if in seen set
        if (date, companyName, text, link) in seen:
            return res

        # add to seen set
        print(title)
        print(datetime.now())
        seen.add(tuple([date, companyName, text, link]))
        print(seen)

        # append to results list
        try:
          res[date][companyName] = {'내용': text, '링크': link}
        except:
          res[date] = {companyName: {'내용': text, '링크': link}} # 해당날짜의 첫 번째 데이터

        return res

    args = [date, companyName, text, link]
    # print(args)

    res = add_entry(res, *args)

    with open('IPODB.json', 'w', encoding='utf-8') as db:
      json.dump(res, db, ensure_ascii=False)

  except:
    pass



  

C:\Users\user\AppData\Local\Temp\ipykernel_8664\2171858672.py:46: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  im.thumbnail((998, 680), Image.ANTIALIAS)


In [139]:
# 그림을 합쳐서 전달하는 방법으로...

title = '투자설명서'
link = 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220916000016'
companyName = '이노룰스'
text = title
date = '2022-09-16'

if ('투자설명서' in title) and not('정정' in title):

  option = Options()
  option.add_argument('--window-size=1920,1080')
  option.add_argument('headless')
  driver = webdriver.Chrome(options=option)
  driver.get(link)
  # time.sleep(2)

  key_word = ('투자위험요소', '사업의 내용')
  dictIPO = {}

  for kw in key_word:
    driver.find_element(By.PARTIAL_LINK_TEXT, kw).click()

    driver.current_url
    html_source = driver.page_source
    bk, kw, ak = html_source.partition(kw)

    rcpNo = re.findall(r'rcpNo\'] = "(.*?)";\n', ak)
    dcmNo = re.findall(r'dcmNo\'] = "(.*?)";\n', ak)
    eleId = re.findall(r'eleId\'] = "(.*?)";\n', ak)
    offset = re.findall(r'offset\'] = "(.*?)";\n', ak)
    length = re.findall(r'length\'] = "(.*?)";\n', ak)
    dtd = re.findall(r'dtd\'] = "(.*?)";\n', ak)

    # url이 있어야 read_html을 통해 table 가져올 수 있다
    frame_url = 'https://dart.fss.or.kr/report/viewer.do?rcpNo=' + rcpNo[0] + '&dcmNo=' + dcmNo[0] + '&eleId=' + eleId[0] + '&offset=' + offset[0] + '&length=' + length[0] + '&dtd=' + dtd[0]
    driver.get(frame_url)

    # html_source = driver.page_source
    table_ifrm = pd.read_html(frame_url, index_col=0, header=0)

    key = kw
    dictIPO[key] = table_ifrm
    driver.get(link)
  
  # print(dictIPO)

  for no in range(len(dictIPO['투자위험요소'])):
    for index, row in dictIPO['투자위험요소'][no].iterrows():
      try:
        if '벤처금융' in index:
          print('벤처금융'+ no)
          tableOverhang = table_ifrm[no]
      except:
        pass

  for no in range(len(dictIPO['사업의 내용'])):
    done = ''
    for index, row in dictIPO['사업의 내용'][no].iterrows():
      if '제품' in index:
        # print('매출유형', no)
        tableSales = table_ifrm[no]
        done = True
        break
      else:
        pass

      # try:
      #   if '제품' in index:
      #     # print('매출유형', no)
      #     tableSales = table_ifrm[no]
      #     done = True
      #     break
      # except:
      #   pass
      
    if done:
      break
  
  dfi.export(tableOverhang, 'images/table_overhang.png')
  im = Image.open('images/table_overhang.png')
  im.thumbnail((998, 680), Image.ANTIALIAS)
  im.save('images/table_overhang.png')
  
  dfi.export(tableSales, 'images/table_sales.png')
  im = Image.open('images/table_sales.png')
  im.thumbnail((998, 680), Image.ANTIALIAS)
  im.save('images/table_sales.png')

  try:
    with open('IPODB.json', encoding='utf-8') as db:
      res = json.load(db)

    seen = set()
    for date in res.keys():
      for company in res[date].keys():
        seen.add((date, company, res[date][company]['내용'], res[date][company]['링크']))
    print(seen)

    # seen = {(date, company, res[date][company]['내용'], res[date][company]['링크']) for date in res.keys() for company in res[date].keys()} # 중첩된 집합 표기법?

    link = 'images/table_overhang.png'

    def add_entry(res, date, companyName, text, link):

        # check if in seen set
        if (date, companyName, text, link) in seen:
            return res

        # add to seen set
        print(title)
        print(datetime.now())
        seen.add(tuple([date, companyName, text, link]))
        print(seen)

        # append to results list
        try:
          res[date][companyName] = {'내용': text, '링크': link}
        except:
          res[date] = {companyName: {'내용': text, '링크': link}} # 해당날짜의 첫 번째 데이터

        return res

    args = [date, companyName, text, link]
    # print(args)

    res = add_entry(res, *args)

    with open('IPODB.json', 'w', encoding='utf-8') as db:
      json.dump(res, db, ensure_ascii=False)

  except:
    pass

{'투자위험요소': [Empty DataFrame
Columns: []
Index: [],                                                                                               용어 설명
용어                                                                                                 
디지털 상품 정보 자동화 시스템 (Digital Product Manager, DPM)  상품의 기준 정보와 업무 규칙을 통합 관리하는 시스템으로 상품 기획, 개발, 릴리스...
디지털 의사결정 자동화 시스템 (Digital Decision Manager, DDM)  디지털 의사결정 자동화 시스템(Digital Decision Manager)은 기업...
디지털 혁신 (Digital Innovation)                         디지털 경제에서 효과적인 조직 운영을 위해 기업의 운영 방향을 재조정하는 것을 의미.
디지털화 (Digitization)                               디지털 기술들과 디지털 정보를 이용하여 일하는 방식, 업무 프로세스의 디지털화를 나...
벤치마킹 테스트 (Benchmarking Test)                        실존하는 비교 대상을 두고 하드웨어나 소프트웨어의 성능을 비교 분석하여 평가하는 것.
비즈니스 룰 (Business Rule)                            특정한 사업적인 면을 정의하고 제한하는 규칙으로서 늘 참이나 거짓으로결정. 사업 규...
업셀 (Up-sell)                                      기존 고객에게 고객이 평소에 사던 상품, 서비스보다 비싼 상품, 서비스를구매하도록 ...
전산화 (Computerization)                            

C:\Users\user\AppData\Local\Temp\ipykernel_8664\1345930951.py:82: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  im.thumbnail((998, 680), Image.ANTIALIAS)
C:\Users\user\AppData\Local\Temp\ipykernel_8664\1345930951.py:87: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  im.thumbnail((998, 680), Image.ANTIALIAS)


In [136]:
# link를 리스트로 만들어서 전달하는 방식으로...

title = '투자설명서'
link = 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220919000002'
companyName = '모델솔루션'
text = title
date = '2002-09-26'

if ('투자설명서' in title) and not('정정' in title):

  option = Options()
  option.add_argument('--window-size=1920,1080')
  option.add_argument('headless')
  driver = webdriver.Chrome(options=option)
  driver.get(link)
  # time.sleep(2)

  key_word = ('투자위험요소', '사업의 내용')
  dictIPO = {}

  for kw in key_word:
    driver.find_element(By.PARTIAL_LINK_TEXT, kw).click()

    driver.current_url
    html_source = driver.page_source
    bk, kw, ak = html_source.partition(kw)

    rcpNo = re.findall(r'rcpNo\'] = "(.*?)";\n', ak)
    dcmNo = re.findall(r'dcmNo\'] = "(.*?)";\n', ak)
    eleId = re.findall(r'eleId\'] = "(.*?)";\n', ak)
    offset = re.findall(r'offset\'] = "(.*?)";\n', ak)
    length = re.findall(r'length\'] = "(.*?)";\n', ak)
    dtd = re.findall(r'dtd\'] = "(.*?)";\n', ak)

    # url이 있어야 read_html을 통해 table 가져올 수 있다
    frame_url = 'https://dart.fss.or.kr/report/viewer.do?rcpNo=' + rcpNo[0] + '&dcmNo=' + dcmNo[0] + '&eleId=' + eleId[0] + '&offset=' + offset[0] + '&length=' + length[0] + '&dtd=' + dtd[0]
    driver.get(frame_url)

    # html_source = driver.page_source
    table_ifrm = pd.read_html(frame_url, index_col=0, header=0)

    key = kw
    dictIPO[key] = table_ifrm
    driver.get(link)

  for no in range(len(dictIPO['투자위험요소'])):
    for index, row in dictIPO['투자위험요소'][no].iterrows():
      try:
        if '벤처금융' in index:
          print('벤처금융'+ no)
          tableOverhang = table_ifrm[no]
      except:
        pass

  for no in range(len(dictIPO['사업의 내용'])):
    done = ''
    for index, row in dictIPO['사업의 내용'][no].iterrows():
      try:
        if '제품' in index:
          # print('매출유형', no)
          tableSales = table_ifrm[no]
          done = True
          break
      except:
        pass
    if done:
      break
  
  dfi.export(tableOverhang, 'images/table_overhang.png')
  im = Image.open('images/table_overhang.png')
  im.thumbnail((998, 680), Image.ANTIALIAS)
  im.save('images/table_overhang.png')
  
  dfi.export(tableSales, 'images/table_sales.png')
  im = Image.open('images/table_sales.png')
  im.thumbnail((998, 680), Image.ANTIALIAS)
  im.save('images/table_sales.png')

  try:
    with open('IPODB.json', encoding='utf-8') as db:
      res = json.load(db)

    seen = set()
    for date in res.keys():
      for company in res[date].keys():
        for i in range(len(res[date][company]['링크'])):
          seen.add((date, company, res[date][company]['내용'], res[date][company]['링크'][i]))
    print(seen)

    # seen = {(date, company, res[date][company]['내용'], res[date][company]['링크']) for date in res.keys() for company in res[date].keys()} # 중첩된 집합 표기법?

    links = ['images/table_overhang.png', 'images/table_sales.png']

    def add_entry(res, date, companyName, text, links):

        # check if in seen set
        # print(links)
        linkList = []
        for link in links:
          print('---------------------------', link)
          print((date, companyName, text, link))
          if (date, companyName, text, link) in seen:
            print('있음')
            linkList.append(link)
            continue
            # return res

          # add to seen set
          print(title)
          print(datetime.now())
          seen.add(tuple([date, companyName, text, link]))
          print(seen)

          # append to results list
          linkList.append(link)
          print(linkList)
          try:
            res[date][companyName] = {'내용': text, '링크': linkList}
          except:
            res[date] = {companyName: {'내용': text, '링크': linkList}} # 해당날짜의 첫 번째 데이터

          print(res)
          return res

    args = [date, companyName, text, links]
    # print(args)

    res = add_entry(res, *args)

    with open('IPODB.json', 'w', encoding='utf-8') as db:
      json.dump(res, db, ensure_ascii=False)

  except:
    print('B')
    pass

C:\Users\user\AppData\Local\Temp\ipykernel_8664\3353759358.py:71: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  im.thumbnail((998, 680), Image.ANTIALIAS)


{('2022-09-06', 'CJ ENM', 'KB Korea Conference 2022', 't'), ('2022-09-06', '파라다이스', '국내 기관투자자와의 커뮤니케이션', 'p'), ('2022-09-06', '파라다이스', '국내 기관투자자와의 커뮤니케이션', 'f'), ('2022-09-06', '파라다이스', '국내 기관투자자와의 커뮤니케이션', '1'), ('2022-09-06', 'CJ ENM', 'KB Korea Conference 2022', ':'), ('2022-09-06', 'CJ ENM', 'KB Korea Conference 2022', 'd'), ('2022-09-06', 'CJ ENM', 'KB Korea Conference 2022', 'N'), ('2022-09-06', '파라다이스', '국내 기관투자자와의 커뮤니케이션', 'm'), ('2022-09-06', '파라다이스', '국내 기관투자자와의 커뮤니케이션', 'k'), ('2022-09-06', '파라다이스', '국내 기관투자자와의 커뮤니케이션', '0'), ('2022-09-06', 'CJ ENM', 'KB Korea Conference 2022', '1'), ('2022-09-06', '모델솔루션', '투자설명서', 'images/table_overhang.png'), ('2022-09-06', 'CJ ENM', 'KB Korea Conference 2022', 'p'), ('2022-09-06', 'CJ ENM', 'KB Korea Conference 2022', 'f'), ('2022-09-06', 'CJ ENM', 'KB Korea Conference 2022', '0'), ('2022-09-06', 'CJ ENM', 'KB Korea Conference 2022', 'm'), ('2022-09-06', 'CJ ENM', 'KB Korea Conference 2022', 'k'), ('2022-09-06', '파라다이스', '국내 기관투자자와의 커뮤니케

C:\Users\user\AppData\Local\Temp\ipykernel_8664\3353759358.py:76: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  im.thumbnail((998, 680), Image.ANTIALIAS)


In [108]:
for no in range(len(dictIPO['사업의 내용'])):
    for index, row in dictIPO['사업의 내용'][no].iterrows():
      try:
        if '제품' in index:
          print(index)
          tableSales = table_ifrm[no]
      except:
        pass

제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품매출
제품매출
제품매출
제품매출
제품매출
제품매출
제품매출
제품매출
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
제품
혁신 제품 발굴, 검토 및 개발


In [110]:
dictIPO['사업의 내용'][2]

""
제조업 벨류체인 분류와 모델솔루션 사업 영역


In [111]:
tableSales

,주요 업무
구분,
"혁신 제품 발굴, 검토 및 개발",- 새로운 아이템 발굴(제품 기획 및 조사) - AR HMD 제품 개발 - 자율...
CM 프로젝트 지원,- CM 프로젝트 연구개발 기능 지원 - CM 엔지니어링 PM - 엔지니어링 솔...
Design,- 자체 제품 개발 인하우스(In-house) 디자인 - 제품 디자인 프로젝트 수...
CMF,"- CMF LAB 운영 - 샘플 제작(CMF샘플, CMF application 제작..."


In [86]:
for no in range(len(dictIPO['투자위험요소'])):
    dictIPO['투자위험요소'][no]
    for index, row in dictIPO['투자위험요소'][no].iterrows():
      # print(index)
      try:
        if '벤처금융' in index:
          print('벤처금융', no)
          tableOverhang = table_ifrm[no]
      except:
        pass


벤처금융 169
벤처금융 169
벤처금융 169
벤처금융 169


In [82]:
tableOverhang

,주 주 명,공모후,공모후.1,유통가능물량,유통가능물량.1,매각제한물량,매각제한물량.1,매각제한물량.2,비고
구 분,,,,,,,,,
구 분,주 주 명,주식수,지분율,주식수,지분율,주식수,지분율,기간,비고
최대주주등,한국타이어앤테크놀로지,4025020,62.92%,-,-,4025020,62.92%,상장일로부터 1년,주2)
최대주주등,소 계,4025020,62.92%,-,-,4025020,62.92%,상장일로부터 1년,-
타인,모네홀딩스,593918,9.28%,-,-,593918,9.28%,상장일로부터 1개월,주3)
타인,소계,593918,9.28%,-,-,593918,9.28%,상장일로부터 1개월,NaN
벤처금융,코너스톤 브릿지 1호 신기술조합,670840,10.49%,-,-,670840,10.49%,상장일로부터 1개월,주4)
벤처금융,코너스톤 시너지 1호 신기술조합,38461,0.60%,-,-,38461,0.60%,상장일로부터 1개월,주4)
벤처금융,코너스톤 윈윈 신기술조합,38461,0.60%,-,-,38461,0.60%,상장일로부터 1개월,주4)
벤처금융,소 계,747762,11.69%,-,-,747762,11.69%,상장일로부터 1개월,주4)


In [42]:
for no in range(len(table_ifrm)):
  for index, row in table_ifrm[no].iterrows():
    try:
      if '벤처금융' in index:
        tableOverhang = table_ifrm[no]
    except:
      pass

tableOverhang

,주 주 명,공모후,공모후.1,유통가능물량,유통가능물량.1,매각제한물량,매각제한물량.1,매각제한물량.2,비고
구 분,,,,,,,,,
구 분,주 주 명,주식수,지분율,주식수,지분율,주식수,지분율,기간,비고
최대주주등,한국타이어앤테크놀로지,4025020,62.92%,-,-,4025020,62.92%,상장일로부터 1년,주2)
최대주주등,소 계,4025020,62.92%,-,-,4025020,62.92%,상장일로부터 1년,-
타인,모네홀딩스,593918,9.28%,-,-,593918,9.28%,상장일로부터 1개월,주3)
타인,소계,593918,9.28%,-,-,593918,9.28%,상장일로부터 1개월,NaN
벤처금융,코너스톤 브릿지 1호 신기술조합,670840,10.49%,-,-,670840,10.49%,상장일로부터 1개월,주4)
벤처금융,코너스톤 시너지 1호 신기술조합,38461,0.60%,-,-,38461,0.60%,상장일로부터 1개월,주4)
벤처금융,코너스톤 윈윈 신기술조합,38461,0.60%,-,-,38461,0.60%,상장일로부터 1개월,주4)
벤처금융,소 계,747762,11.69%,-,-,747762,11.69%,상장일로부터 1개월,주4)


In [47]:
dfi.export(tableOverhang, 'images/table_overhang.png')

In [40]:
table_ifrm[77].index

Int64Index([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021], dtype='int64', name='연도')

In [14]:
pprint(html_source)

('<html style="border:0"><head>\n'
 '<title></title>\n'
 '<meta http-equiv="X-UA-Compatible" content="ie=edge">\n'
 '<meta http-equiv="X-UA-TextLayoutMetrics" content="gdi">\n'
 '<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n'
 '<link rel="stylesheet" type="text/css" href="/css/report_xml.css">\n'
 '</head>\n'
 '<body bgcolor="#FFFFFF">\n'
 '<p><br></p>\n'
 '<p class="section-1"><a name="toc1">III. 투자위험요소</a></p>\n'
 '<p><br>당사는 최고 수준의 CNC 가공 기술과 CNC 가공물의 품질을 높이는 후공정 기술, 그리고 고품질의 신속 금형 제작 기술을 '
 '기반으로 다양한 고부가가치 전방산업의 제품 개발 단계에서 필요로 하는 Design 프로토타입, Functional 프로토타입, 사출성형 '
 '프로토타입 및 소량생산 서비스를 공급하는 사업을 영위하고 있습니다. 프로토타입은 제조업 프로세스 상 제작 목적과 타입, 부가가치 등을 '
 '기준으로 아래와 같이 구분할 수 있습니다. 당사가 영위하는 사업과 관련된 주요 용어들의 정의는 다음과 같습니다.<br><br></p>\n'
 '<table class="nb" width="700">\n'
 '<colgroup>\n'
 '<col width="700">\n'
 '</colgroup>\n'
 '<tbody>\n'
 '<tr>\n'
 '  <td width="700" height="20" align="CENTER">[주요 용어 해설]</td>\n'
 '</tr>\n'
 '</tbody>\n'
 '</table>\n'
 '<table border="1

In [8]:
pprint(html_source)

('<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="ko" lang="ko" class=" '
 'ext-strict"><head>\n'
 '\t<title>\n'
 '\t모델솔루션/투자설명서/2022.09.19\n'
 '\t</title>\n'
 '\t<meta charset="UTF-8">\n'
 '\t<meta name="viewport" content="width=device-width, user-scaleable=no">\n'
 '\t<meta http-equiv="X-UA-Compatible" content="ie=edge">\n'
 '\t\n'
 '\t\n'
 '\n'
 '\n'
 '\t\n'
 '<script type="text/javascript" '
 'src="/resource/js/jquery-3.3.1.min.js"></script>\n'
 '\n'
 '<link rel="stylesheet" href="/js/jquery-ui/jquery-ui.min.css">\n'
 '<script type="text/javascript" '
 'src="/js/jquery-ui/jquery-ui.min.js"></script>\n'
 '\n'
 '<!-- 2011.11.01 ext 2.3 -->\n'
 '<!--[if lte IE 8]><link rel="stylesheet" type="text/css" '
 'href="/js/ext-main/resources/css/ext-all-ie8.css" /><![endif]-->\n'
 '<script type="text/javascript" '
 'src="/js/ext-main/adapter/ext/ext-base.js"></script>\n'
 '<script type="text/javascript" src="/js/ext-main/ext-all.js"></script>\n'
 '\n'
 '<!-- x-xeries js libraries  -->\n'